# Example of using `cells_in_gel`

## Imports

In [ ]:
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd
import numpy as np
import glob

from skimage.morphology import disk

from cells_in_gel import preprocess as pp
from cells_in_gel import properties as props
from cells_in_gel import batch

## Load Images

Load images from a filepath using `glob.glob('filenames.tif')` into a list of strings. 

In [ ]:
# list of filenames
files = glob.glob('C3*.tif') 

Using the `cells_in_gel.batch.max_projection(files)`, input the list of filenames to create a dictionary of max intensity projections. 

In [ ]:
max_proj = batch.max_projection(files)

Here is a plot showing the max intensity projection from the 19th filename in the list of `files`.

In [ ]:
plt.imshow(max_proj[files[18]])

## Segmentation and Region Properties

Create the `selem` for separation of objects. A higher number will result in less separation between objects in max intensity projections. (Need to update our original preprocess function to optimize this a bit better).

In [ ]:
selem = disk(0)

Using `cells_in_gel.batch.labels_regions(files)`, segment max projections, label objects, and extract region properties. The output `regions` will give a dictionary containing dataframes of properties for each max intensity projection. 

In [ ]:
regions = batch.labels_regions(files, selem, min_size=350)

Here is the head for the first max intensity projection dataframe in our list of `files`.

In [ ]:
regions[files[0]].head()

## Extracting Average Values from Max Intesnity Projections

Create a list of `titles` that describe categories between each underscore in filenames. Create a list of average `properties` to extract from the max intensity projection regions dataframe.

In [ ]:
titles = ['celltype', 'ecmtype_conc', 'rgd_conc', 'mag', 'img_num']
properties = ['area', 'eccentricity', 'extent']

Using `cells_in_gel.batch.image_summaries(files, properties, titles, regions)`, extract a summary dataframe that includes average and variance properties from each max intensity projection. See the `summary` head below. 

In [ ]:
summary = batch.image_summaries(files, properties, titles, regions)
summary.head()

Further analysis of grouping, etc. will be done using the `summary` dataframe. At this point, you can use `summary.to_csv('imagesummaries_green.csv', index=False)` for creating a CSV of this dataframe.

In [ ]:
summary.to_csv('imagesummaries_green_2.csv')

## WORK IN PROGRESS: Graphing Groups

List category headers, cell types, ecm types, rgd/no rgd. Next, groupby unique combinations of all three categories. 

In [ ]:
categ = ['celltype', 'ecmtype_conc', 'rgd_conc']
cells = ['C3-I61QTTA-CFbs', 'C3-NTG-CFbs']
ecms = ['I61Q1ECM', 'I61Q5ECM', 'NTG1ECM', 'NTG5ECM', 'NoECM']
rgds = ['0mMRGD', '1mMRGD']

group_names = []
# create empty dictionary where group_names are the keys for each group
groups = {}


for rgd in rgds:
    for cell in cells:
        for ecm in ecms:
            name = cell + '_' + ecm + '_' + rgd
            group_names.append(name)
            groups[name] = summary.groupby(categ).get_group((cell, ecm, rgd))

Here is an example of a single group consisting of NTG fibroblasts, I61Q5ECM, 1mMRGD.

In [ ]:
groups[group_names[16]]

Create list of columns from each group, and create a new dataframe with average from each group!

In [ ]:
columns = ['area', 'area_var', 'eccentricity', 'eccentricity_var', 'extent', 'extent_var']

tot = []

for name in group_names:
    gr = []
    for column in columns:
        gr.append(groups[name][column].mean())
    tot.append(gr)

fin = pd.DataFrame(tot, group_names, columns)

Here is an example of what the final dataframe looks like, one row for each group - wahoo!

In [ ]:
final = fin.reset_index()
final

Labels will be the groups on each graph (ECM type). Create lists of each cell type (I6QTTA, NTG) properties (area, eccentricity, extent).

In [ ]:
ecms = ['I61Q1ECM', 'I61Q5ECM', 'NTG1ECM', 'NTG5ECM', 'NoECM']
labels = ecms
properties = ['area', 'eccentricity', 'extent']

i6_norgd = []  # empty list of I6QTTA no rgd group
i6_rgd = []  # empty list of I6QTTA rgd group
ntg_norgd = []  # empty list of NTG no rgd group
ntg_rgd = []  # empty list of NTG rgd group

# get all properties from group
for prop in properties:
    i6_norgd.append(final[prop][0:5].values.tolist())
    i6_rgd.append(final[prop][10:15].values.tolist())
    
    ntg_norgd.append(final[prop][5:10].values.tolist())
    ntg_rgd.append(final[prop][15:20].values.tolist())

# example of property 0 : area for I6QTTA cells with no rgd (corresponding to all 5 ecm types)
i6_norgd[0]

Graphing each group!

In [ ]:
x = np.arange(len(labels))  # the label locations
width = 0.35  # the width of the bars

fig, axes = plt.subplots(ncols=2, nrows=3, figsize=(14, 13))

# plotting I6QTTA mean cell area
axes[0, 0].bar(x - width/2, i6_norgd[0], width, label='No RGD') # without RGC
axes[0, 0].bar(x + width/2, i6_rgd[0], width, label='RGD') # with RGD
axes[0, 0].set_ylabel('Mean Cell Area (pixels)')
axes[0, 0].set_title('I61QTTA Fibroblasts')

# NTG mean cell area
axes[0, 1].bar(x - width/2, ntg_norgd[0], width, label='No RGD')
axes[0, 1].bar(x + width/2, ntg_rgd[0], width, label='RGD')
axes[0, 1].set_ylabel('Mean Cell Area (pixels)')
axes[0, 1].set_title('NTG Fibroblasts')

# I6QTTA eccentricity
axes[1, 0].bar(x - width/2, i6_norgd[1], width, label='No RGD')
axes[1, 0].bar(x + width/2, i6_rgd[1], width, label='RGD')
axes[1, 0].set_ylabel('Eccentricity')
axes[1, 0].set_title('I61QTTA Fibroblasts')

# NTG eccentricity
axes[1, 1].bar(x - width/2, ntg_norgd[1], width, label='No RGD')
axes[1, 1].bar(x + width/2, ntg_rgd[1], width, label='RGD')
axes[1, 1].set_ylabel('Eccentricity')
axes[1, 1].set_title('NTG Fibroblasts')

# I61QTTA extent
axes[2, 0].bar(x - width/2, i6_norgd[2], width, label='No RGD')
axes[2, 0].bar(x + width/2, i6_rgd[2], width, label='RGD')
axes[2, 0].set_ylabel('Extent')
axes[2, 0].set_title('I61QTTA Fibroblasts')

# NTG extent
axes[2, 1].bar(x - width/2, ntg_norgd[2], width, label='No RGD')
axes[2, 1].bar(x + width/2, ntg_rgd[2], width, label='RGD')
axes[2, 1].set_ylabel('Extent')
axes[2, 1].set_title('NTG Fibroblasts')

for ax in axes.flatten():
    ax.set_xticks(x)
    ax.set_xticklabels(labels)
    ax.legend()

fig.tight_layout()

plt.show()